In [2]:
from utils.data_preprocessing import *
import os

In [3]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed')
csv_data = load_csv_files(base_path)

In [4]:
gr_data = {}

for well, w_dict in csv_data.items():
    gr_data[well] = {}

    print(f"Arquivos de {well} que possuem GR:")
    for logical_file, lf_dict in w_dict.items():
        gr_data[well][logical_file] = {}

        for frame, df in lf_dict.items():
            try:
                gr_df = df[['GR', 'TDEP']]

                gr_data[well][logical_file][frame] = gr_df

                print(logical_file + frame)
            except:
                pass
    
    print("\n")

Arquivos de 1CSC 0001  SE que possuem GR:
logical_file_0frame_0.csv
logical_file_10frame_0.csv
logical_file_11frame_0.csv
logical_file_12frame_0.csv


Arquivos de 1IDA 0001  SE que possuem GR:
logical_file_5frame_0.csv


Arquivos de 1IV  0002  SE que possuem GR:
logical_file_0frame_0.csv
logical_file_4frame_0.csv
logical_file_7frame_0.csv
logical_file_8frame_0.csv


Arquivos de 1MO  0001  SE que possuem GR:
logical_file_7frame_0.csv




In [5]:
max_tdeps = {}

for well, w_dict in gr_data.items():
    max_tdep = float('-inf')
    
    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            try:
                # Remove lines with NaN
                df.dropna(inplace=True)

                max_tdep = max(max_tdep, df['TDEP'].max())
            except:
                pass

    max_tdeps[well] = max_tdep    
    

C:\Users\rafae\AppData\Local\Temp\ipykernel_31308\3344277069.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_31308\3344277069.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_31308\3344277069.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Lo

In [21]:
def find_next_df(w_dict, ref):
    min_diff = float('inf')
    closest_df = None
    closest_value = None

    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            try:
                # Calcula o valor mais próximo da referência no dataframe atual
                first_tdep = df['TDEP'].min()
                    
                # Verifica se este valor é o mais próximo da referência até agora
                if abs(first_tdep - ref) < min_diff:
                    min_diff = abs(first_tdep - ref)
                    closest_df = w_dict[logical_file][frame]
            except:
                pass

    return closest_df

In [29]:
def find_df_between(df_ref, w_dict):
    ref_first_tdep = df_ref['TDEP'].min()
    ref_last_tdep = df_ref['TDEP'].max()

    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            try:
                first_tdep = df['TDEP'].min()
                last_tdep = df['TDEP'].max()

                if (
                    (ref_first_tdep < first_tdep < ref_last_tdep) and 
                    (last_tdep > ref_last_tdep)
                ):
                    return df
            except:
                pass
            
    return None

In [30]:
spliced_data = {}
curr_min_tdep = {}

for well, w_dict in gr_data.items():
    curr_min_tdep[well] = 0
    curr_df = None
    
    while(curr_min_tdep[well] != max_tdeps[well]):
        if curr_df is None:
            curr_df = find_next_df(w_dict, curr_min_tdep[well])
        elif find_df_between(curr_df, w_dict) is None:
            curr_df = find_next_df(w_dict, curr_min_tdep[well])
        else:
            curr_df = find_df_between(curr_df, w_dict)
        
        curr_min_tdep[well] = curr_df['TDEP'].max()
        print(f"{well}: {curr_min_tdep[well]}")

1CSC 0001  SE: 1376.0
1CSC 0001  SE: 1378.8
1CSC 0001  SE: 2814.0
1CSC 0001  SE: 2814.8
1IDA 0001  SE: 3480.4
1IV  0002  SE: 1846.0
1IV  0002  SE: 1854.0
1IV  0002  SE: 3051.0
1IV  0002  SE: 3060.0
1MO  0001  SE: 2817.8
